# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [ ]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

### 1.1 Building vocabulary

#### Code

In [1]:
import numpy as np
from collections import defaultdict

# So, build the model based on make the vocabulary set on training set
f = open("./data/lm/train.txt",'r+',encoding="utf-8")
lines = []
s = set(["<s>","</s>","<UNK>"])
sentences = []
for line in f.readlines()[:-1]:
    # some lines are still end with \n, need to remove \n
    if line.endswith("\n"):
        line = line[:-1]
    tmp = line.split(' ')
    sentences.append(tmp)
    for word in tmp:
        s.add(word)
f.close()

word_list = sorted(list(s))

word_count = defaultdict(int)


# count every word
for sen in sentences:
    for word in sen:
        word_count[word]+=1

# calculate <UNK>
word_count_dic = {"<UNK>":0}
for key in word_list:
    value = word_count[key]
    if value<3:
        word_count_dic["<UNK>"]+=1
    else:
        word_count_dic[key]=value

word_list = sorted(word_count_dic.keys())
word_dic = {key:idx for (idx,key) in enumerate(word_list)}
tmp = []
for key in word_dic:
    tmp.append(word_count_dic[key])
word_count = np.array(tmp)

# change sentence to id list

tmp_sentences = []
for sentence in sentences:
    tmp = []
    for word in sentence:
        if word in word_dic:
            tmp.append(word_dic[word])
        else:
            tmp.append(word_dic["<UNK>"])
    tmp_sentences.append(tmp[:])
sentences = tmp_sentences

#vocabulary size
print("vocabulary size: %d"%len(word_list))

del word_list
del word_count_dic
del tmp


# word_dic : the map between words and id, 
# word_count : the number of each word organized with id
# sentences : sentences in training data

vocabulary size: 22629


#### Discussion

Please show the vocabulary size and discuss the number of parameters of n-gram models.

The vocabulary size is 22,629, not including `<s>` and `</s>` characters. So, as for the number of parameters of n-gram models, when the n increases, the number of parameters increases sharply. For example, when n is 1 in this case, the number of parameters is 22,629 which is the vocabulary size. When n is 2, the vocabulary size is 512,071,641 which is the square of vocabulary size. For n equals 3, the vocabulary size is the cube of vocabulary size : 11,587,669,164,189. If we use a float variable to store a single probability, it would use 86,334.86303827912GB memory. So, the bigger n we have, more memory would be used by probability matrix. 

### 1.2 $n$-gram Language Modeling

After preparing your vocabulary, you are expected to build bigram and unigram language models and report their perplexity on the training set, and dev set. Please discuss your experimental results. If you encounter any problems, please analyze them and explain why.

#### Code

In [2]:
from nltk.util import bigrams
from collections import Counter
import math

##### Model Definition

In [3]:
# unigram language model
# as for n=1, I only need to calculate every words' count and then calculate the probability matrix
# which is quite easy to write
# the probability matrix would look like this:
#           [P_0, P_1, P_2, P_3, ...... , P_n, P_start, P_end]
#           P_start, P_end are the probability of start padding and end padding

class UnigramModel:
    def __init__(self, sentences,word_dic,word_count,file = ""):
        if file!="":
            self.uniProb = np.load(file)
        else:
            self.uniProb = self.__cal_prob(sentence,word_dic,word_count)
        self.word_dic = word_dic
        
    # calculate the probability array
    def __cal_prob(self,sentences,word_dic,word_count):
        startpad, endpad = len(sentences),len(sentences)
        count = startpad+endpad
        for nu in word_count:
            count += nu

        uniProb = [0.0] * (len(word_dic)+2)
        for i in range(len(word_dic)):
            uniProb[i] = word_count[i] / count
        uniProb[-2] = startpad / count
        uniProb[-1] = endpad / count
        return np.array(uniProb)

In [1]:
# run the model and save the results
# uni = UnigramModel(sentences,word_dic,word_count)
# np.save("./unigram.npy",uni.uniProb)

In [4]:
uni = UnigramModel(sentences,word_dic,word_count,file="./unigram.npy")

In [4]:
# bigram language model
# the probability matrix would look like this:
#    first word(i) /second word(j)
#           [w_0, w_1, w_2, w_3, ...... , w_n, startpad, endpad]
#     [w_0]
#     ......
#     [startpad]
#     [endpad] this line is meaningless, deleted
#                       matrix[i][j] = P(j|i)

class BigramModel:
    def __init__(self,sentences,word_dic,word_count,file = ""):
        if file != "":
            self.biProb = np.load(file)
        else:
            # self.uniProb = UnigramModel(sentences,word_dic,word_count).uniProb
            self.biProb = self.__cal_prob(sentences,word_dic,word_count)
        self.word_dic = word_dic
    
    # calculate the probability matrix
    def __cal_prob(self,sentences,word_dic,word_count):
        count_matrix = np.zeros((len(word_dic)+1,len(word_dic)+2),dtype=float)
        startpad,endpad = len(word_dic),len(word_dic)+1
        for sen in sentences:
            for (before,after) in bigrams(sen,pad_left=True, pad_right=True, left_pad_symbol=startpad, right_pad_symbol=endpad):
                count_matrix[before][after] += 1
        
        # add <s> at the end of word_count, which is equal to the number of sentences
        word_count = np.append(word_count,np.array([len(sentences)]))
        for i in range(len(count_matrix)):
            for j in range(len(count_matrix[0])):
                count_matrix[i][j] /= word_count[i]
        return count_matrix



In [ ]:
# calculate the model and save it to disk
# bi = BigramModel(sentences,word_dic,word_count)    

# np.save("./bigram.npy",bi.biProb)

In [7]:
# read model from file
bi = BigramModel(sentences,word_dic,word_count,"./bigram.npy")

##### calculate perplexity 

In [5]:
# calculate ppl of uni-gram model
def uni_ppl(un:UnigramModel):
    f = open("./data/lm/test.txt","r",encoding="utf-8")
    su = 0.0
    unkidx = word_dic["<UNK>"]
    M = 0
    for line in f.readlines():
        M+=1
        # cur state at <s> which is -1 index
        cur = -1
        p = 0.0
        fail = False
        for word in line.split(" "):
            if word=="\n":
                continue
            if word in word_dic:
                su += math.log2(un.uniProb[word_dic[word]])
            else:
                su += math.log2(un.uniProb[unkidx])
                
        
        # for </s> multiply this probability
        su += math.log2(un.uniProb[-1])

    f.close()

    l = su/M
    # print("this is l",l)
    ppl = math.pow(2,-l)
    print("the perplexity of this uni-gram model is:%f"%ppl)

In [6]:
un = UnigramModel(sentences,word_dic,word_count,"./unigram.npy")
uni_ppl(un)

the perplexity of this uni-gram model is:1859724089293349491953848083657071635917363617064062578726577401507349738291200.000000


In [8]:
# calculate perplexity of bi-gram model
import string


def bi_ppl(bi:BigramModel, filePath:string):
    f = open(filePath,"r",encoding="utf-8")
    su = 0.0
    unkidx = word_dic["<UNK>"]
    M = 0
    for line in f.readlines():
        M += 1
        # cur state at <s> which is -1 index
        cur = -1
        # p = 1.0
        p = 0.0
        fail = False
        for word in line.split(" "):
            if word == "\n":
                continue
            if word in word_dic :
                nextstep = word_dic[word]
                # p *= bi.biProb[cur][nextstep]
                if bi.biProb[cur][nextstep] != 0:
                    p += math.log2(bi.biProb[cur][nextstep])
                    cur = nextstep
                else:
                    # if test fail, break 
                    fail = True
                    break
            else:
                if bi.biProb[cur][unkidx] != 0:
                    # p *= bi.biProb[cur][unkidx]
                    p += math.log2(bi.biProb[cur][unkidx])
                    cur = unkidx
                else:
                    # if test fail, break 
                    fail = True
                    break
        if not fail:
            p += math.log2(bi.biProb[cur][-1]) # for </s> multiply this probability
            su += p
        else:
            su += float("-inf")
    f.close()
    l = su/M
    ppl = math.pow(2,-l)
    print("the perplexity of this bi-gram model is:%f"%ppl)

In [9]:
bi = BigramModel(sentences,word_dic,word_count,"./bigram.npy")
bi_ppl(bi,"./data/lm/test.txt")

the perplexity of this bi-gram model is:inf


As for the first version of the program, p mutipled a lot times which leads to p equal to 0. As for the result, in the second version of the program, I change the way to calculate su.

sometimes, biProb[i][j]==0, whichi means test fail, at that time perplexity of this sentence would be float('-inf').

#### Discussion

##### Problem Encountered

During this process, I meet 2 diffient problems both related to how to calculate the perplexity. The one problem is occurred when calculate $p(x^{(i)})$, where $x^{(i)}$ is the $i^{th}$ sentence in the test set. The other problem happens when calculate the perplexity of bi-gran model.

+ Question 1

    The main issue of the first problem is mainly related to the represent ability of a float number, because Python uses float type to store decimal numbers. When a dicimal number is smaller than 0.00000001, the result of this number would be zero and this would lead to value error when I run log2 function. To resolve this problem, I use another way to calculate the perplexity. The definition of perplexity is listed below:
    $$ l = {{{1} \over {M}}  \sum^{m}_{i=1} \log_{2}{p(x^{(i)})}}  $$
    $$ ppl = 2^{-l} $$
    $$ {{p(x^{(i)})} = \prod_{j=0}^n p(w_j|w_{j-1})} \text{\quad where $w_j$ is the $j^{th}$ word in $sentence_i$, n=len($sentence_i$)} $$ 
    Because if I calculate $p(x^{(i)})$ roughly, $p(x^{(i)})$ would be too small to store in a float type number. So I decide to use this   function to calculate $l$:
    $$ l = {{{1} \over {M}}  \sum^{m}_{i=1} \sum^{n}_{j=0} \log_{2}{p(w_j|w_{j-1})}} $$
    $$\text{\quad where $w_j$ is the $j^{th}$ word in $sentence_i$, $w_{-1}$="<s>", n=len($sentence_i$)},m=len(sentences) $$
    With this function, the result of calculating perplexity would not be too small, so that the perplexity can be calculated successfully. 

+ Question 2 

    As for the second problem, it occored when $p(w_j|w_{j-1})$ equals 0 during testing bi-gram model. The mainly reason could be that the train set is not large enough to make sure that every possible $p(w_j|w_{j-1})$ is included. If a word set ($w_{j-1}$,$w_j$) is met and the probability $p(w_j|w_{j-1})$ equals zero, then the test prosess would be terminated because there is no probability to generate a sentence like the test sentence. 

    In this case, the $l$ should be self defigned in the program because calculating $\log_{2}0$ is actually an error mathmatically. So in the actual program, when then program test a sentence failed because $p(w_j|w_{j-1})=0$, it is better to add a punishment to the attribute $l$, I choose to add $-\infty$ as the result of $\sum^{n}_{j=0} \log_{2}{p(w_j|w_{j-1})}$ because $\lim_{x\rightarrow{0}}\log_{2}x=-\infty$.

##### Result Discussion

The perplexity of uni-gram model is 1859724089293349491953848083657071635917363617064062578726577401507349738291200.000000, and the perplexity of the bi-gram model is inf.

As for the uni-gram model, the perplexity is quight high, which means this model cannot predict the sentences in the test set. I think the main reason is that this model only is contained the probability of each words and every word is generated in a certain probability. So that the perplexity is high.

In [10]:
le,wid = len(bi.biProb),len(bi.biProb[0])
t = le*wid
for i in range(le):
    for j in range(wid):
        if bi.biProb[i][j]!=0:
            t-=1
print("the probability is 0 occupied: %f%%"%(t/le/wid*100))

the probability is 0 occupied: 99.901397%


As for the bi-gram model, the perplexity is infinite, which means this model is terrible.  Actually, in the probability matrix generated by the bi-gram model, nearly 99.9% of the conditional probability are 0. This would caused by the size of training set, if the training set is not large enough, it is possible that the probability matrix calculated by the bi-gram model could not contains every possible two word set in the real world. Therefore, the perplexity could be infinite.

However, I believe that there is some ways that can improve this model. Firstly, increasing the size of training data. Secondly, using smoothing to give little probability to every zero members in the probability matrix. Finally, combining the bi-gram model with the uni-gram model.

### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

In [6]:
class AddOneBigramModel(BigramModel):
    def __init__(self,sentences,word_dic,word_count,file = ""):
        super(AddOneBigramModel, self).__init__(sentences,word_dic,word_count,file)
    
    # with add-one smoothing
    def __cal_prob(self,sentences,word_dic):
        # for the one in add one smoothing
        count_matrix = np.ones((len(word_dic)+1,len(word_dic)+2),dtype=float)
        print(count_matrix[0][0])
        startpad,endpad = len(word_dic),len(word_dic)+1
        count = 0
        for sen in sentences:
            for (before,after) in bigrams(sen,pad_left=True, pad_right=True, left_pad_symbol=startpad, right_pad_symbol=endpad):
                count_matrix[before][after] += 1
                count += 1
        V = (len(word_dic)+1)*(len(word_dic)+2)
        for i in range(len(count_matrix)):
            for j in range(len(count_matrix[0])):
                if count_matrix[i][j] != 0:
                    count_matrix[i][j] /= (count+V)
        return count_matrix


In [7]:
AddOneSmoothing = AddOneBigramModel(sentences,word_dic,word_count)


##### Discussion

##### Optional: Add-k smoothing

###### Code

###### Discussion

#### 1.3.2 Linear Interpolation

##### Code

##### Discussion

##### Optional: Optimization

###### Discussion

## 2 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out